# Tutorial 5: Word Embedding: Mejorar la representación vectorial de los textos tomando en cuenta las relaciones semánticas

## 1. Motivación: representar las dimensiones semánticas de cada palabra

1. I want an orange juice.
2. I want an apple ____ .

- Los enfoques <i>bag of words</i> no tienen la capacidad de calcular que las frases 1 y 2 son muy similares porque no tienen una manera de representar que las palabras 'orange' y 'apple' comparten caracterícas (<i>features</i>) comunes.

Los enfoques ingenuos tieden a representar las palabras como vectores "1-Hot". Por ejemplo, supongamos que tenemos un vocabulario de sólo cinco palabras: King, Queen, Man, Woman y Child. Se codificaría la palabra 'Queen' como:

<img src="img/word2vec1.png"/>

- Sería más interesante poder representar la semántica de cada palabra tomando en cuentas ciertas características. 

<img src="img/word2vec2.png"/>


## 2 Definición

El concepto de **word embedding** se refiere a un conjunto de técnicas utilizadas para aprender representaciones matemáticas, tipicamente vectores, de cada palabra.

Una de las técnicas más populares es __Word2Vec__ propuesto por un equipo de investigación de Google en 2013 (Efficient Estimation of Word Representations in Vector Space [Mikolov et al., 2013]).

Alternativas populares son __GloVe__ (propuesta por la Universidad de Stanford en 2014) y __FastText__ (propuesta por Facebook en 2016), que extende Word2Vec para considerar de mejor manera las palabras con errores ortográficas.

## 3 Algunas propiedades de los word embeddings

- Tener representaciones vectoriales de las palabras permite calcular "razonamiento" de tipo __King - Man + Woman = ?__ y llegar a un resultado cerca de __Queen__.

<img src="img/word2vec4.png"/>

- Tener representaciones vectoriales de las palabras permite realizar razonamientos analógicos de tipo __A es a B, lo que C es a ..__ . Este tipo de propiedades es muy útil para aplicaciones de _Question Answering_ por ejemplo. Las respuestas a las pregutas siguientes <i>¿Cuál es la capital de Chile?</i> o <i>¿Cuáles son los clubs de fútbol en Chile?</i> se pueden responder adicionando vectores.

<img src="img/word2vec6.png"/>

<img src="img/word2vec7.png"/>

<img src="img/word2vec8.png"/>

Para construir sus vectores, Word2Vec utiliza un dataset de entrenamiento y algoritmos de aprendizaje basados en redes neuronales (__Continuous Bag of Words__ (CBOW), o modelo __Skip Gram__). El objetivo de esta fase de aprendizaje es aprender cuáles son las palabras _X_ más probables de aparecer en el contexto de una palabra _y_.

<img src="img/word2vec5.png"/>

Por ejemplo, ¿cuál es la probabilidad de tener la palabra 'perro' si aparece la palabra 'pelota' en el contexto?

<code>Los expertos explican que los __perros__ persiguen __pelotas__ en movimiento como parte de un comportamiento instintivo. Aunque no todos los perros tienen tan despiertos su instinto de caza, esto no impide que la mayoría de ellos sí disfruten, y mucho, de los juegos que incluyen persecuciones de una saltarina __pelota__ que bota delante de ellos. </code>

__Algoritmo CBOW__

Las palabras de contexto forman la capa de entrada. Si el tamaño del vocabulario es V, estos serán vectores de dimensión V con sólo uno de los elementos establecido en uno, y el resto todos los ceros. Hay una sola capa oculta y una capa de salida.

<img src="img/word2vec9.png"/>

## 4. Ejemplo de Word Embedding en Python (con Gensim)

La clase <code>word2vec</code> de Gensim permite word embeddings de palabras (ver documentación: https://radimrehurek.com/gensim/models/word2vec.html).

Esta clase tiene varios parametros, en particular:
- <code>sentences</code>: una lista de palabras o de frases que sirve para entrenar el modelo
- <code>sg</code>: define que algoritmos de aprendizaje utilizar (0=CBOW, 1=skip-gram)
- <code>size</code>: define la dimensión de los vectores que se desea extraer
- <code>window</code>: define el número de palabras considerar a la izquierda y a la derecha de una palabra
- <code>min_count</code>: ignorar las palabras que aparecen menos de _min_count_
y otros asociados a la parametrización de la fase de aprendizaje de la red neuronal (que no detallaremos en esta parte del curso):
- <code>alpha</code>: el _learning rate_ utilizado para optimizar los parametros de la red neuronal.
- <code>iter</code>: número de iteraciones (epocas) sobre el dataset para encontrar los parametreos que optimizan la red neuronal.

In [3]:
from gensim.models import word2vec

In [10]:
sentences = word2vec.Text8Corpus('datos/text8.txt')

In [11]:
model = word2vec.Word2Vec(sentences,size=200,hs=1)
#model=word2vec.Word2Vec.load("text8_model")

In [12]:
model.save("text8_model")
model=word2vec.Word2Vec.load("text8_model")

In [13]:
print(model)

Word2Vec(vocab=71290, size=200, alpha=0.025)


In [14]:
model.wv.most_similar(positive=['woman','king'],negative=['man'],topn=5)

[('queen', 0.5620756149291992),
 ('throne', 0.5534422397613525),
 ('regent', 0.5253190994262695),
 ('isabella', 0.5218691229820251),
 ('empress', 0.5112246870994568)]

In [15]:
model.wv.most_similar(positive=["conflict"])

[('clashes', 0.6951584815979004),
 ('conflicts', 0.6921702027320862),
 ('confrontation', 0.6602897644042969),
 ('hostilities', 0.6406719088554382),
 ('struggle', 0.6317054033279419),
 ('tensions', 0.6200854778289795),
 ('confrontations', 0.6106060147285461),
 ('disputes', 0.601070761680603),
 ('dispute', 0.5880026817321777),
 ('strife', 0.5856019258499146)]

In [16]:
model.wv.most_similar(positive=["conflict","weapon"])

[('confrontation', 0.6138920783996582),
 ('warfare', 0.5560401082038879),
 ('weapons', 0.529255211353302),
 ('fighting', 0.5263172388076782),
 ('conflicts', 0.5184081792831421),
 ('pistol', 0.512715220451355),
 ('warheads', 0.5096693634986877),
 ('assault', 0.4955317974090576),
 ('clashes', 0.4931451380252838),
 ('struggle', 0.49076008796691895)]

In [17]:
model.wv.most_similar(positive=["conflict"],negative=["weapon"])

[('clashes', 0.4917221665382385),
 ('conflicts', 0.45864203572273254),
 ('disagreements', 0.4525034427642822),
 ('disputes', 0.4470274746417999),
 ('tensions', 0.43743643164634705),
 ('hostilities', 0.4324764609336853),
 ('reconciliation', 0.4302550256252289),
 ('negotiations', 0.42687636613845825),
 ('dispute', 0.422853022813797),
 ('strife', 0.41570529341697693)]

In [18]:
model.wv.most_similar(positive=["life"])

[('childhood', 0.5351325273513794),
 ('lives', 0.5025249123573303),
 ('career', 0.49570488929748535),
 ('experiences', 0.47748303413391113),
 ('teens', 0.4545215666294098),
 ('genius', 0.4330507516860962),
 ('work', 0.4302922785282135),
 ('humanity', 0.4295467138290405),
 ('intellect', 0.4159848690032959),
 ('adolescence', 0.41503143310546875)]

In [19]:
model.wv.most_similar(positive=["life"],negative=["money"])

[('childhood', 0.3839266002178192),
 ('recounting', 0.36995333433151245),
 ('experiences', 0.35863131284713745),
 ('incarnations', 0.33248966932296753),
 ('incarnation', 0.321285605430603),
 ('retelling', 0.3173801898956299),
 ('adolescence', 0.31677621603012085),
 ('conception', 0.31466156244277954),
 ('lives', 0.3073475956916809),
 ('manifestation', 0.30052706599235535)]

Ver los parametros aprendidos por la red neuronal para una palabra dada:

In [20]:
model.wv['computer']

array([-5.1952833e-01,  1.1148246e-01,  6.9580138e-01, -2.5740319e-01,
        4.7186688e-01,  1.0890626e+00,  7.8428811e-01,  1.2575428e+00,
       -3.2528302e-01, -5.9351665e-01, -2.0826530e+00,  3.5133177e-01,
       -9.4672424e-01,  2.2377522e+00,  2.5126532e-01,  9.9040347e-01,
        4.0315992e-01, -1.3104205e+00,  1.2499611e+00, -3.7213158e-02,
        3.6723572e-01,  6.5381400e-02, -6.4698070e-01, -8.3988988e-01,
       -1.7005359e-01, -6.4206636e-01, -4.9831229e-01,  1.4915508e-01,
        1.5017015e+00,  1.1041881e+00,  4.4298092e-01, -8.4363335e-01,
        5.3172308e-01,  2.0636346e+00, -9.7007346e-01,  1.5372264e-01,
       -2.2156003e+00,  1.2702472e+00,  1.4431497e+00,  8.2726759e-01,
        1.1144599e+00, -2.8155036e-02, -6.1116415e-01,  1.0096577e+00,
        1.2654163e-03,  7.8814006e-01, -2.5798957e+00, -2.3802340e+00,
       -1.4076350e+00,  1.7053335e+00,  7.3089552e-01,  5.7254595e-01,
       -5.8497757e-01,  3.4365246e-01,  1.1702532e+00, -3.4459218e-01,
      

In [21]:
model.wv.doesnt_match("breakfast cereal dinner lunch".split())

/home/mvernier/miniconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'cereal'

In [22]:
model.wv.doesnt_match("brazil chile france peru argentina".split())

'france'

In [23]:
model.wv.doesnt_match("apple pear banana hammer".split())

'hammer'

In [24]:
model.wv.similarity('man','hammer')

0.22704087

In [25]:
model.wv.similarity('woman','hammer')

0.14314103

In [26]:
model.wv.similarity('man','engineer')

0.14349055

In [27]:
model.wv.similarity('woman','engineer')

0.07078395

In [28]:
model.wv.similarity('man','baby')

0.33093974

In [29]:
model.wv.similarity('woman','baby')

0.4592277

### 5. Limitaciones

Las técnicas de Word Embeddings dan resultados muy interesantes pero tienen dos principales limitaciones:


1) **Capturan los estereotipos culturales**. Puede generar problemas éticos de sesgos y injusticias, si se hace una confianza ciega en los modelos predictivos.


2) No permiten tomar en cuenta que ciertas **palabras cambian de significado según el contexto**.

Ejemplo: "I lost my computer __mouse__"


3) No permiten tomar en cuenta **el orden entre las palabras**.

Ejemplo: "Estamos aqui para trabajar y no jugar" vs. "Estamos aqui para jugar y no trabajar"



Para mejorar estas limitaciones:

- Utilizar modelos predictivos que toman en cuenta el contexto de las palabras

- Utilizar modelos predictivos que toman en cuenta el orden entre las palabras

--> **Redes neuronales convolucionales** (CNN) y **Redes neuronales transformers** (BERT, GPT-2, GPT-3, etc.)





